# new

In [1]:
import os
os.chdir("/home/joseph/Coding/ml_projects/g2net_ml_dl/python")
import g2net
os.chdir("../")
from g2net.utils.config_reader import load_config
cfg_path = "/home/joseph/Coding/ml_projects/g2net_ml_dl/python/scripts/test_infer.yml"
cfg = load_config(cfg_path)["infer_config"]
cfg

{'dset_dir': '/home/joseph/datasets/g2net_train',
 'export_dir': '/home/joseph/datasets/g2net_train/preds',
 'threshold': 0.5,
 'idx_start': 30000,
 'idx_end': 30006,
 'batch_size': 2,
 'num_workers': 8,
 'cpu_only': True,
 'base_model_paths': ['/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/base/fold0.pt',
  '/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/base/fold1.pt',
  '/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/base/fold2.pt',
  '/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/base/fold3.pt',
  '/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/base/fold4.pt'],
 'filter_model_paths': ['/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/filter/logs_0/best.pth',
  '/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/filter/logs_1/best.pth',
  '/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/filter/logs_2/best.pth',
  '/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/filter/logs_3/best.pth',
  '/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/filt

In [2]:
from g2net.inference import Inferrer, create_test_transforms, create_test_dataloader
import pandas as pd
test_path = os.path.join(cfg["dset_dir"], "train.csv")
start = cfg["idx_start"]
# end = start+2
end = cfg["idx_end"]
test_df = pd.read_csv(test_path).iloc[start:end]

transforms = create_test_transforms()
test_loader = create_test_dataloader(test_df,
                                         batch_size=2,
                                         test_transforms=transforms)
inferrer = Inferrer(test_loader,
                  cfg["base_model_paths"],
                  cfg["filter_model_paths"],
                  cfg["export_dir"],
                  threshold=cfg["threshold"],
                  cpu_only=True)
len(inferrer.test_loader)

Tracking  ['fold', 'base_time', 'filter_time', 'both_time']
/home/joseph/datasets/g2net_train/preds/base_preds and subdirs already exist!
/home/joseph/datasets/g2net_train/preds/filter_preds and subdirs already exist!
/home/joseph/datasets/g2net_train/preds/both_preds and subdirs already exist!
Initialized Inferrer...


3

In [4]:
inferrer.infer_all()

Running inference for fold 0
Loading default minirocket params...
Running inference for fold 1
Running inference for fold 2
Running inference for fold 3
Running inference for fold 4
Results: 
      fold                 base_time               filter_time  \
0       0    0 days 00:00:04.451305    0 days 00:00:00.990848   
1       1    0 days 00:00:04.975408    0 days 00:00:01.386699   
2       2    0 days 00:00:05.057856    0 days 00:00:01.397900   
3       3    0 days 00:00:04.886840    0 days 00:00:01.310305   
4       4    0 days 00:00:04.726790    0 days 00:00:01.281028   
5    mean 0 days 00:00:04.819639800    0 days 00:00:01.273356   
6  stddev 0 days 00:00:00.239696770 0 days 00:00:00.165523942   

                  both_time  
0    0 days 00:00:04.252171  
1    0 days 00:00:04.426650  
2    0 days 00:00:04.725288  
3    0 days 00:00:02.468100  
4    0 days 00:00:03.146022  
5 0 days 00:00:03.803646200  
6 0 days 00:00:00.955887686  
Inference results: 
      fold                

In [5]:
from torchsummary import summary

summary(inferrer.base_model, (3, 24))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 128, 6]          24,704
            Conv1d-2               [-1, 128, 6]           6,272
            Conv1d-3               [-1, 128, 6]           1,664
    CNNSpectrogram-4            [-1, 3, 128, 6]               0
          Identity-5            [-1, 3, 128, 6]               0
        Conv2dSame-6            [-1, 56, 64, 3]           1,512
       BatchNorm2d-7            [-1, 56, 64, 3]             112
              SiLU-8            [-1, 56, 64, 3]               0
            Conv2d-9            [-1, 56, 64, 3]             504
      BatchNorm2d-10            [-1, 56, 64, 3]             112
             SiLU-11            [-1, 56, 64, 3]               0
           Conv2d-12             [-1, 14, 1, 1]             798
             SiLU-13             [-1, 14, 1, 1]               0
           Conv2d-14             [-1, 5

In [6]:
from torchsummary import summary

summary(inferrer.filter_model, (3, 24))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
MiniRocketFeatures-1                 [-1, 9996]               0
           Flatten-2                 [-1, 9996]               0
       BatchNorm1d-3                 [-1, 9996]          19,992
            Linear-4                    [-1, 1]           9,997
Total params: 29,989
Trainable params: 29,989
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.23
Params size (MB): 0.11
Estimated Total Size (MB): 0.34
----------------------------------------------------------------


In [6]:
from g2net.inference import GlobalEvaluator
from glob import glob

num_folds = len(cfg["base_model_paths"])
base_model_paths = {fold:glob(os.path.join(cfg["export_dir"], "base_preds", f"fold{fold}", "*.npy")) for fold in range(num_folds)}
filter_preds_paths = {fold:glob(os.path.join(cfg["export_dir"], "filter_preds", f"fold{fold}", "*.npy")) for fold in range(num_folds)}
both_preds_paths = {fold:glob(os.path.join(cfg["export_dir"], "both_preds", f"fold{fold}", "*.npy")) for fold in range(num_folds)}

evaluator = GlobalEvaluator(test_loader, base_model_paths,
                            filter_preds_paths, both_preds_paths)
all_auc = evaluator.evaluate_all()
all_auc

Tracking  ['fold', 'base_auc', 'filter_auc', 'both_auc']
Evaluator initialized... 
Results: 
      fold  base_auc  filter_auc  both_auc
0       0  0.000000     0.37500  0.125000
1       1  0.000000     0.75000  0.000000
2       2  0.000000     0.62500  0.250000
3       3  0.125000     0.75000  0.750000
4       4  0.000000     0.75000  0.250000
5    mean  0.025000     0.65000  0.275000
6  stddev  0.055902     0.16298  0.285044
Evaluation results: 
      fold  base_auc  filter_auc  both_auc
0       0  0.000000     0.37500  0.125000
1       1  0.000000     0.75000  0.000000
2       2  0.000000     0.62500  0.250000
3       3  0.125000     0.75000  0.750000
4       4  0.000000     0.75000  0.250000
5    mean  0.025000     0.65000  0.275000
6  stddev  0.055902     0.16298  0.285044


# Actual

In [1]:
import pandas as pd

# times = pd.read_csv("/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/inference_times.csv").drop(["Unnamed: 0"])
times = pd.read_csv("/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/inference_times.csv").drop(columns=["Unnamed: 0"])
times

,fold,base_time,filter_time,both_time
0,0,0 days 00:01:06.167713,0 days 00:00:28.248476,0 days 00:01:09.816149
1,1,0 days 00:01:05.424859,0 days 00:00:27.847005,0 days 00:01:08.946569
2,2,0 days 00:01:05.832677,0 days 00:00:28.031748,0 days 00:01:08.495583
3,3,0 days 00:01:05.648685,0 days 00:00:28.197953,0 days 00:01:08.341410
4,4,0 days 00:01:07.007622,0 days 00:00:27.793906,0 days 00:01:10.267162
5,mean,0 days 00:01:06.016311200,0 days 00:00:28.023817600,0 days 00:01:09.173374600
6,stddev,0 days 00:00:00.617313910,0 days 00:00:00.203088321,0 days 00:00:00.838531027


In [10]:
times["base_time"].iloc[0][7:]

'00:01:06.167713'

In [2]:
aucs = pd.read_csv("/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/inference_metrics.csv").drop(columns=["Unnamed: 0"])
aucs

,fold,base_auc,filter_auc,both_auc
0,0.0,0.846669,0.624791,0.722213
1,1.0,0.844374,0.619141,0.726964
2,2.0,0.844771,0.621648,0.714124
3,3.0,0.846055,0.616073,0.706928
4,4.0,0.850198,0.612998,0.740806
5,mean,0.846413,0.618930,0.722207
6,stddev,0.002311,0.004614,0.012917


# old

In [ ]:
#   dset_dir: "${HOME}/datasets/g2net_train"
#   export_dir: "${HOME}/datasets/g2net_train"
    # base_model_paths:
  #   - "/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/base/fold0.pt"
  #   - "/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/base/fold1.pt"
  #   - "/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/base/fold2.pt"
  #   - "/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/base/fold3.pt"
  #   - "/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/base/fold4.pt"
  # filter_model_paths:
  #   - "/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/filter/logs_0/best.pth"
  #   - "/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/filter/logs_1/best.pth"
  #   - "/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/filter/logs_2/best.pth"
  #   - "/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/filter/logs_3/best.pth"
  #   - "/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/filter/logs_4/best.pth"


In [1]:
import os
os.chdir("/home/joseph/Coding/ml_projects/g2net_ml_dl/python")
import g2net
os.chdir("../")
from g2net.utils.config_reader import load_config
cfg_path = "/home/joseph/Coding/ml_projects/g2net_ml_dl/python/scripts/test_infer.yml"
cfg = load_config(cfg_path)["infer_config"]
cfg

{'dset_dir': '/home/joseph/datasets/g2net_train',
 'threshold': 0.5,
 'idx_start': 30000,
 'idx_end': 30003,
 'batch_size': 512,
 'num_workers': 8,
 'base_model_paths': ['/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/base/fold0.pt',
  '/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/base/fold1.pt',
  '/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/base/fold2.pt',
  '/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/base/fold3.pt',
  '/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/base/fold4.pt'],
 'filter_model_paths': ['/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/filter/logs_0/best.pth',
  '/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/filter/logs_1/best.pth',
  '/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/filter/logs_2/best.pth',
  '/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/filter/logs_3/best.pth',
  '/home/joseph/Coding/ml_projects/g2net_ml_dl/weights/filter/logs_4/best.pth']}

In [2]:
from g2net.inference import Inferrer, create_test_transforms, create_test_dataloader
import pandas as pd
test_path = os.path.join(cfg["dset_dir"], "train.csv")
start = cfg["idx_start"]
end = start+2
test_df = pd.read_csv(test_path).iloc[start:end]

transforms = create_test_transforms()
test_loader = create_test_dataloader(test_df, batch_size=2)
inferrer = Inferrer(test_loader,
                  cfg["base_model_paths"],
                  cfg["filter_model_paths"],
                  threshold=cfg["threshold"],
                  cpu_only=True)
len(inferrer.test_loader)

Tracking  ['fold', 'base_auc', 'base_time', 'mr_auc', 'mr_time', 'both_auc', 'both_time']


1

In [3]:
inferrer.infer_all()

Running inference for fold 0
Loading default minirocket params...
Running inference for fold 1
Running inference for fold 2
Running inference for fold 3
Running inference for fold 4
Results: 
      fold  base_auc                 base_time  mr_auc  \
0       0       0.5    0 days 00:00:01.724612     0.5   
1       1       0.5    0 days 00:00:03.038530     0.5   
2       2       0.5    0 days 00:00:02.703668     0.5   
3       3       0.5    0 days 00:00:02.694826     0.5   
4       4       0.5    0 days 00:00:02.532766     0.5   
5    mean       0.5 0 days 00:00:02.538880400     0.5   
6  stddev       0.0 0 days 00:00:00.490966933     0.0   

                    mr_time  both_auc                 both_time  
0    0 days 00:00:00.560213       0.5    0 days 00:00:00.748936  
1    0 days 00:00:00.693553       0.5    0 days 00:00:00.964562  
2    0 days 00:00:00.887760       0.5    0 days 00:00:00.782747  
3    0 days 00:00:00.709914       0.5    0 days 00:00:00.790654  
4    0 days 00:00:00

In [4]:
inferrer.metrics

,fold,base_auc,base_time,mr_auc,mr_time,both_auc,both_time
0,0,0.5,0 days 00:00:01.724612,0.5,0 days 00:00:00.560213,0.5,0 days 00:00:00.748936
1,1,0.5,0 days 00:00:03.038530,0.5,0 days 00:00:00.693553,0.5,0 days 00:00:00.964562
2,2,0.5,0 days 00:00:02.703668,0.5,0 days 00:00:00.887760,0.5,0 days 00:00:00.782747
3,3,0.5,0 days 00:00:02.694826,0.5,0 days 00:00:00.709914,0.5,0 days 00:00:00.790654
4,4,0.5,0 days 00:00:02.532766,0.5,0 days 00:00:00.721942,0.5,0 days 00:00:00.829530
5,mean,0.5,0 days 00:00:02.538880400,0.5,0 days 00:00:00.714676400,0.5,0 days 00:00:00.823285800
6,stddev,0.0,0 days 00:00:00.490966933,0.0,0 days 00:00:00.116553144,0.0,0 days 00:00:00.084014992


In [1]:
!python /home/joseph/Coding/ml_projects/g2net_ml_dl/python/scripts/infer.py --yml_path="/home/joseph/Coding/ml_projects/g2net_ml_dl/python/scripts/test_infer.yml"

Traceback (most recent call last):
  File "/home/joseph/Coding/ml_projects/g2net_ml_dl/python/scripts/infer.py", line 5, in <module>
    from g2net.utils.config_reader import load_config
ModuleNotFoundError: No module named 'g2net'


# Unit Tests

In [3]:
inferrer.infer_minirocket_only(0)

Creating minirocket model...
Loading default minirocket params...


{'mr_auc': 0.5, 'mr_time': datetime.timedelta(microseconds=498950)}

In [4]:
inferrer.infer_minirocket_only(1)

Creating minirocket model...


{'mr_auc': 0.5, 'mr_time': datetime.timedelta(microseconds=512292)}

In [5]:
inferrer.infer_base_only(0)

{'base_auc': 0.5,
 'base_time': datetime.timedelta(seconds=1, microseconds=468996)}

In [6]:
inferrer.infer_both(inferer.filter_model, inferer.base_model)

{'both_auc': 0.5, 'both_time': datetime.timedelta(microseconds=748334)}

In [7]:
inferrer.infer_single_fold(0)

Creating minirocket model...


{'mr_auc': 0.5,
 'mr_time': datetime.timedelta(microseconds=717335),
 'base_auc': 0.5,
 'base_time': datetime.timedelta(seconds=2, microseconds=224812),
 'both_auc': 0.5,
 'both_time': datetime.timedelta(microseconds=774901)}

In [9]:
inferrer.infer_all()

Running inference for fold 0
Creating minirocket model...
Running inference for fold 1
Creating minirocket model...
Running inference for fold 2
Creating minirocket model...
Running inference for fold 3
Creating minirocket model...
Running inference for fold 4
Creating minirocket model...
Results: 
      fold  base_auc                 base_time  mr_auc  \
0       0       0.5    0 days 00:00:02.464728     0.5   
1       1       0.5    0 days 00:00:02.043293     0.5   
2       2       0.5    0 days 00:00:02.224947     0.5   
3       3       0.5    0 days 00:00:02.592533     0.5   
4       4       0.5    0 days 00:00:02.619959     0.5   
5  stddev       0.0 0 days 00:00:00.248459828     0.0   
6     NaN       NaN                       NaT     NaN   

                    mr_time  both_auc                 both_time  
0    0 days 00:00:00.765671       0.5    0 days 00:00:00.956732  
1    0 days 00:00:00.731449       0.5    0 days 00:00:00.711574  
2    0 days 00:00:00.632981       0.5    0 d

In [10]:
inferrer.metrics

,fold,base_auc,base_time,mr_auc,mr_time,both_auc,both_time
0,0,0.5,0 days 00:00:02.464728,0.5,0 days 00:00:00.765671,0.5,0 days 00:00:00.956732
1,1,0.5,0 days 00:00:02.043293,0.5,0 days 00:00:00.731449,0.5,0 days 00:00:00.711574
2,2,0.5,0 days 00:00:02.224947,0.5,0 days 00:00:00.632981,0.5,0 days 00:00:00.711218
3,3,0.5,0 days 00:00:02.592533,0.5,0 days 00:00:00.723470,0.5,0 days 00:00:00.782677
4,4,0.5,0 days 00:00:02.619959,0.5,0 days 00:00:00.726942,0.5,0 days 00:00:00.778170
5,stddev,0.0,0 days 00:00:00.248459828,0.0,0 days 00:00:00.049430531,0.0,0 days 00:00:00.100413933
6,NaN,NaN,NaT,NaN,NaT,NaN,NaT


In [22]:
pred.shape, pos_idx.shape, pred[pos_idx].shape

(torch.Size([2, 1]), torch.Size([2, 1]), torch.Size([0]))

In [53]:
pred = s(inferrer.filter_model(batch[0]))
p = pred.detach().cpu().numpy()
p[0] = 1
p.shape, p

((2, 1),
 array([[1.       ],
        [0.0010824]], dtype=float32))

In [86]:
def use_pos_batches(x: torch.Tensor, batches_mask: torch.Tensor):
    pos_idx = [
        batch_idx for batch_idx, boolean in enumerate(batches_mask)
        if boolean
    ]
    return x[pos_idx]

In [88]:
use_pos_batches(x, pos_idx).shape

torch.Size([1, 3, 4096])

In [65]:
pos_idx = pred > 0.5
batch[0][pos_idx, :,]

IndexError: The shape of the mask [2, 1] at index 1 does not match the shape of the indexed tensor [2, 3, 4096] at index 1

In [78]:
pos_idx[:, :, None].repeat(1, 3, 4096).shape

torch.Size([2, 3, 4096])

In [84]:
for b in pos_idx:
    print(b)

tensor([True])
tensor([False])


In [80]:
masked = torch.ones(x.shape) * pos_idx[:, :, None].repeat(1, 3, 4096)
masked.shape

torch.Size([2, 3, 4096])

In [67]:
x = batch[0]
x.shape

torch.Size([2, 3, 4096])

In [62]:
if neg_idx.any():
    print("Trueee")

Trueee


In [55]:
neg_idx = pred < 0.5
pred[neg_idx] = 0
pred

tensor([[1.],
        [0.]], grad_fn=<IndexPutBackward>)

In [60]:
pred.any()

RuntimeError: isDifferentiableType(variable.scalar_type())INTERNAL ASSERT FAILED at "../torch/csrc/autograd/functions/utils.h":65, please report a bug to PyTorch. 

In [56]:
len(pred < 0.5)

2

In [51]:
import numpy as np
p[p < 0.5] = 0.5
p

array([[1. ],
       [0.5]], dtype=float32)

In [59]:
(p < 0.5).any()

True

In [28]:
import torch
s = torch.nn.Sigmoid()
s(pred)

tensor([[0.0011],
        [0.0011]], grad_fn=<SigmoidBackward>)

In [29]:
inferer.filter_model

MiniRocket(
  (backbone): MiniRocketFeatures()
  (head): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): BatchNorm1d(9996, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=9996, out_features=1, bias=True)
  )
)

In [32]:
inferer.base_model

SpectroCNN(
  (spectrogram): CNNSpectrogram(
    (spec_conv): ModuleList(
      (0): Conv1d(3, 128, kernel_size=(64,), stride=(4,), padding=(31,))
      (1): Conv1d(3, 128, kernel_size=(16,), stride=(4,), padding=(7,))
      (2): Conv1d(3, 128, kernel_size=(4,), stride=(4,), padding=(1,))
    )
  )
  (cnn): Sequential(
    (0): Identity()
    (1): EfficientNet(
      (conv_stem): Conv2dSame(3, 56, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn1): BatchNorm2d(56, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act1): SiLU(inplace=True)
      (blocks): Sequential(
        (0): Sequential(
          (0): DepthwiseSeparableConv(
            (conv_dw): Conv2d(56, 56, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=56, bias=False)
            (bn1): BatchNorm2d(56, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (act1): SiLU(inplace=True)
            (se): SqueezeExcite(
              (conv_reduce): Conv2d(56, 14, kernel_

In [34]:
p2 = inferer.base_model(batch[0])
s = torch.nn.Sigmoid()
s(p2)

tensor([[0.1226],
        [0.1226]], grad_fn=<SigmoidBackward>)

In [35]:
batch[-1]

tensor([[1.],
        [0.]])